In [1]:
from ucimlrepo import fetch_ucirepo 

In [ ]:
#N -> Nominal Data
#O -> Ordinal Data
#D -> Discrete Data
#C -> Continous Data
data_type = {"age":"C" 
            ,"sex":"N" #Binomial
            ,"cp": "N"
            ,"trestbps": "C"
            ,"chol": "C"
            ,"fbs": "N"
            ,"restecg": "N"
            ,"thalach": "C"
            ,"exang": "N" #Binomial
            ,"oldpeak": ""
            ,"slope":
            ,"ca":
            ,"thal":}

In [2]:
# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 

# data (as pandas dataframes) 
X = heart_disease.data.features 
y = heart_disease.data.targets

In [3]:
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,63,1,1,145,233,1,2,150,0,2.3,3,0.0,6.0
1,67,1,4,160,286,0,2,108,1,1.5,2,3.0,3.0
2,67,1,4,120,229,0,2,129,1,2.6,2,2.0,7.0
3,37,1,3,130,250,0,0,187,0,3.5,3,0.0,3.0
4,41,0,2,130,204,0,2,172,0,1.4,1,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,45,1,1,110,264,0,0,132,0,1.2,2,0.0,7.0
299,68,1,4,144,193,1,0,141,0,3.4,2,2.0,7.0
300,57,1,4,130,131,0,0,115,1,1.2,2,1.0,7.0
301,57,0,2,130,236,0,2,174,0,0.0,2,1.0,3.0
